# Aemet OpenData
Aemet makes available to the interested public meteorological data from some of its stations through the OpenData service (https://opendata.aemet.es/centrodedescargas/inicio).

Aemet allows you to make as many data requests as you need, but there are some limitations per request.:
* If all stations are requested, the maximum date range is 31 days.
* If data from some stations are requested, one request per station must be made.
* For each station, a maximum of 5 years for daily data and 36 months for monthly data can be requested.

The module automates the download of meteorological time series for long time periods and for more than one station. Internally, it decomposes the request into multiple requests that satisfy Aemet constraints. For each internal request to the Aemet server, a csv file is saved. Once all the files have been downloaded, they can be stored in a Sqlite3 database and exported to a single csv file.

To start using the service you have to create a free key in (https://opendata.aemet.es/centrodedescargas/inicio) and save it in the apikey.txt file. 

## Data and Metadata
For each data request, you can download the data, the metadata or both. 

Information provided by metadata.
* Monthly meteorological data. All columns are of type string. The data containing data of real type (float) must be deduced from the examination of the downloaded contents.
* Daily data. Columns are of type float or string. In the float columns the separator is the comma. The daily precipitation column, in spite of being of float type, has string values to denote special values: 1) Ip = less than 0.1 mm: 2) Acum = Accumulated precipitation.

Consider that the original format of the downloaded data is json. For each request, the metadata corresponds to the data available in the request. It may happen that the columns downloaded in different requests are different, as some columns are optional.


In [1]:
from datetime import date

from aemet_open_data import AemetOpenData

## User-supplied parameters.
The methods used to download data require some values to be supplied by the user. Not all methods require the same parameters.

* __Parameters__
* d1 (date): Data will be requested between date d1 and date d2; d1 is the start date
* d2 (date): Data will be requested between date d1 and date d2; d2 is the end date
* dir_path (str): directory path where files will be saved
* fetch (str): For a set of variables, controls whether data, metadata or both will be downloaded; a value in ('data', 'metadata' or 'both')
* stations (list of str): List of meteorological stations from which data is requested
* time_step (str): Temporal resolution of meteorological data to be downloaded: 'day' or 'month'
* use_files (bool), optional, default True: If True data saved in previously downloaded files not be requested again 
* verbose (bool), optional, default True: Controls the number of messages in the screen. If True all messages will be displayed

The notebook is organized in pairs of cells:
* In the first one the parameters are given values.
* In the second cell the corresponding method is executed

## Meteorological stations in OpenData

In [3]:
# parameters
dir_path = './download'  
fetch = 'both'  
use_files = True  
verbose = True  

In [4]:
aod = AemetOpenData()
aod.meteo_stations(dir_path, fetch, use_files, verbose)

estaciones_open_data_data.csv: 200, OK 
estaciones_open_data_metadata.csv: 200, OK Inventario de estaciones para el apartado Valores Climatología


## Daily meteorological data from all stations
When you request data from all stations, Aemet only provides daily data.

In [5]:
# parameters
d1 = date(2023, 1, 1)
d2 = date(2023, 8, 1)
dir_path = './download'
fetch ='both'  
verbose = True  
use_files = True  

In [6]:
aod = AemetOpenData()
downloaded_files = aod.meteo_data_all_stations(d1, d2, dir_path, fetch, verbose, use_files)
print('Downloaded files: ', len(downloaded_files))

stations_20230101T000000UTC_20230201T235959UTC_data.csv: 200, OK 
stations_20230101T000000UTC_20230201T235959UTC_metadata.csv: 200, OK Climatologías diarias
stations_20230202T000000UTC_20230305T235959UTC_data.csv: 200, OK 
stations_20230202T000000UTC_20230305T235959UTC_metadata.csv: 200, OK Climatologías diarias
stations_20230306T000000UTC_20230406T235959UTC_data.csv: 200, OK 
stations_20230306T000000UTC_20230406T235959UTC_metadata.csv: 200, OK Climatologías diarias
stations_20230407T000000UTC_20230508T235959UTC_data.csv: 200, OK 
stations_20230407T000000UTC_20230508T235959UTC_metadata.csv: 200, OK Climatologías diarias
stations_20230509T000000UTC_20230609T235959UTC_data.csv: 200, OK 
stations_20230509T000000UTC_20230609T235959UTC_metadata.csv: 200, OK Climatologías diarias
stations_20230610T000000UTC_20230711T235959UTC_data.csv: 200, OK 
stations_20230610T000000UTC_20230711T235959UTC_metadata.csv: 200, OK Climatologías diarias
stations_20230712T000000UTC_20230801T235959UTC_data.csv: 2

## Daily or monthly meteorological data from selected stations

In [7]:
#parameters
time_step = 'day'
d1 = date(2019, 1, 1)
d2 = date(2019, 3, 31)
stations = ['7178I', '7031', '7031X']
dir_path = './download'
fetch = 'both'
verbose = True
use_files = True

In [8]:
aod = AemetOpenData()
downloaded_files = aod.meteo_data_by_station(time_step, d1, d2, stations, dir_path, fetch, verbose, use_files)
print('Downloaded files: ', len(downloaded_files))

7178I_20190101T000000UTC_20190331T235959UTC_data.csv: 200, OK 
7178I_20190101T000000UTC_20190331T235959UTC_metadata.csv: 200, OK Climatologías diarias
7031_20190101T000000UTC_20190331T235959UTC_data.csv: 200, OK 
7031_20190101T000000UTC_20190331T235959UTC_metadata.csv: 200, OK Climatologías diarias
7031X_20190101T000000UTC_20190331T235959UTC_data.csv: 200, OK 
7031X_20190101T000000UTC_20190331T235959UTC_metadata.csv: 200, OK Climatologías diarias
Downloaded files:  6


In [9]:
# Now I download monthly data
time_step = 'month'

aod = AemetOpenData()
downloaded_files = aod.meteo_data_by_station(time_step, d1, d2, stations, dir_path, fetch, verbose, use_files)
print('Downloaded files: ', len(downloaded_files))

7178I_2019_2019_data.csv: 200, OK 
7178I_2019_2019_metadata.csv: 200, OK Climatologías mensuales anuales
7031_2019_2019_data.csv: 200, OK 
7031_2019_2019_metadata.csv: 200, OK Climatologías mensuales anuales
7031X_2019_2019_data.csv: 200, OK 
7031X_2019_2019_metadata.csv: 200, OK Climatologías mensuales anuales
Downloaded files:  6


## Export downloaded meteorological data files to a Sqlite3 database 
If the data request to Aemet generated many files, the AOD_2db module can be used to concatenate them into one.
The module saves the data in a Sqlite3 database; once the table is created it can be exported to a csv file.<br>
A specific database is created according to how the data request was made to the Aemet server: 
* Daily weather data from all stations.
* Daily weather data from selected stations.
* Monthly weather data from selected stations.
Each database has only one table.

In [1]:
from aod_2db import AOD_2db

### Create a database

In [2]:
# parameters        
dpath = './download'  # directory with csv files
"""
type of request to Aemet: 
* 'stations_day'. Daily meteorological data from all stations
* 'station1_day'. Daily meteorological data from selected stations
* 'station1_month'. Monthly meteorological data from selected stations
"""
ftype = 'stations_day'  

In [3]:
a2db = AOD_2db(dpath, ftype)
if not a2db.to_db():
    print('Attemp failed')


C:\Users\solis\Documents\DEV\python3\meteoro\aemet_open_data\download\metd_all_stations.db
already exists, overwrite (y/n)?:  y



Previous downloaded files has been imported to the sqlite db
C:\Users\solis\Documents\DEV\python3\meteoro\aemet_open_data\download\metd_all_stations.db


### Export database to a csv file


In [4]:
# parameters
dpath = './download'  # directory path where the database is stored
ftype = 'stations_day'
point_dec_sep = True  # if database has daily data and point_dec_sep is True, decimal sep. is point

In [5]:
a2db = AOD_2db(dpath, ftype)
a2db.to_db(point_dec_sep)


C:\Users\solis\Documents\DEV\python3\meteoro\aemet_open_data\download\metd_all_stations.db
already exists, overwrite (y/n)?:  y


0 stations_20230101T000000UTC_20230201T235959UTC_data.csv
1 stations_20230202T000000UTC_20230305T235959UTC_data.csv
2 stations_20230306T000000UTC_20230406T235959UTC_data.csv
3 stations_20230407T000000UTC_20230508T235959UTC_data.csv
4 stations_20230509T000000UTC_20230609T235959UTC_data.csv
5 stations_20230610T000000UTC_20230711T235959UTC_data.csv
6 stations_20230712T000000UTC_20230801T235959UTC_data.csv

Previous downloaded files has been imported to the sqlite db
C:\Users\solis\Documents\DEV\python3\meteoro\aemet_open_data\download\metd_all_stations.db
Updated decimal separator as "." in columns: velmedia, tmin, prec, tmax, altitud, sol, racha, dir, tmed, presmin, presmax


True

In [6]:
a2db.to_csv()


C:\Users\solis\Documents\DEV\python3\meteoro\aemet_open_data\download\metd_all_stations.csv exists, continue (y/n): ?  y


Data dumped to
C:\Users\solis\Documents\DEV\python3\meteoro\aemet_open_data\download\metd_all_stations.csv


True